In [1]:
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')


In [2]:
from llama_index import set_global_handler

set_global_handler("simple")

In [3]:
from bellek.llama_index.indices.knowledge_graph.base import patch_knowledge_graph_index
from bellek.llama_index.data_structs.data_structs import patch_kg_data_struct

patch_kg_data_struct()
patch_knowledge_graph_index()


In [4]:
import json
from pathlib import Path

import kuzu
import typer
from dotenv import load_dotenv
from llama_index import ServiceContext, StorageContext, load_index_from_storage
from llama_index.callbacks import CallbackManager
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.indices.base import BaseIndex
from llama_index.llms import OpenAI
from llama_index.query_engine.retriever_query_engine import RetrieverQueryEngine
from rich.console import Console

from bellek.llama_index.graph_stores.kuzu import KuzuGraphStore
from bellek.llama_index.obs import make_phoenix_trace_callback_handler
from bellek.utils import set_seed

err = Console(stderr=True).print

load_dotenv()

set_seed(42)


2024-02-17 21:23:35,610 - phoenix.datasets.dataset - INFO - Dataset: phoenix_dataset_c73b62d8-5b56-402c-8ad3-7e25df5ac223 initialized


In [5]:
# model to generate embeddings for triplets
embed_model = HuggingFaceEmbedding("sentence-transformers/all-MiniLM-L6-v2")

# language model to use for triplet extraction
llm = OpenAI(temperature=0, model="gpt-3.5-turbo-1106")


def make_service_context(directory: Path, example_id: str):
    # example_dir = directory / example_id
    # example_dir.mkdir(exist_ok=True, parents=True)

    # traces_filepath = example_dir / "traces.jsonl"
    # traces_filepath.unlink(missing_ok=True)
    # trace_callback_handler = make_phoenix_trace_callback_handler(traces_filepath)
    # callback_manager = CallbackManager(handlers=[trace_callback_handler])
    return ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        # callback_manager=callback_manager,
    )

def load_storage_context(directory: Path):
    db = kuzu.Database(str(directory / "kuzu"))
    graph_store = KuzuGraphStore(db)
    storage_context = StorageContext.from_defaults(persist_dir=directory / "index", graph_store=graph_store)
    return storage_context

def load_index(directory: Path, service_context: ServiceContext):
    storage_context = load_storage_context(directory)
    return load_index_from_storage(
        storage_context,
        service_context=service_context,
        include_embeddings=True,
    )

/Users/bdsaglam/dev/repos/bellek/.venv/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [6]:
TEXT_QA_PROMPT_USER_MSG_CONTENT = """Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer in 2-4 words: """



def make_query_engine(index: BaseIndex):
    query_engine: RetrieverQueryEngine = index.as_query_engine(
        include_text=False,
        embedding_mode="hybrid",
        response_mode="simple_summarize",
        verbose=True,
    )
    original_text_qa_prompt = query_engine.get_prompts()["response_synthesizer:text_qa_template"]
    original_text_qa_prompt.conditionals[0][1].message_templates[1].content = TEXT_QA_PROMPT_USER_MSG_CONTENT
    query_engine.update_prompts({"response_synthesizer:text_qa_template": original_text_qa_prompt})
    return query_engine

In [7]:
def answer_questions(query_engine, example):
    sub_questions = [item["question"] for item in example["question_decomposition"]]
    hop1_question = sub_questions[0]
    hop1_answer = query_engine.query(hop1_question).response
    example["question_decomposition"][0]["answer"] = hop1_answer
    hop2_question = sub_questions[1].replace("#1", hop1_answer)
    hop2_answer = query_engine.query(hop2_question).response
    example["question_decomposition"][1]["answer"] = hop2_answer
    return example


In [8]:
dataset_file = Path("../../data/generated/musique-kg-llm/train/question-decomposition.jsonl")
knowledge_graph_directory = Path("../../data/generated/musique-kg-llm/train/knowledge-graphs/")
out = Path("../../data/generated/musique-kg-llm/train/answers.jsonl")

In [9]:
lines = dataset_file.read_text().strip().split("\n")

In [10]:
line = lines[57]

In [11]:
intermediate_directory = dataset_file.parent / "question-answering"
intermediate_directory.mkdir(exist_ok=True, parents=True)

example = json.loads(line)
example_id = example["id"]

err(f"Setting up query engine for {example_id}")
service_context = make_service_context(intermediate_directory, example_id)
index = load_index(knowledge_graph_directory / example_id, service_context)
query_engine = make_query_engine(index)

err(f"Answering the question in the sample {example_id}")
example_answered = answer_questions(query_engine, example)


Setting up query engine for 2hop__354073_511430

2024-02-17 21:23:56,870 - llama_index.indices.loading - INFO - Loading all indices.


Answering the question in the sample 2hop__354073_511430

2024-02-17 21:23:56,968 - httpx - INFO - HTTP Request: POST http://localhost:8111/chat/completions "HTTP/1.1 200 OK"
2024-02-17 21:23:57,062 - bellek.llama_index.indices.knowledge_graph.retrievers - WARNING - Index was not constructed with embeddings, skipping embedding usage...


** Messages: **
user: A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.
---------------------
In which country is Debel located?
---------------------
Provide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'

**************************************************
** Response: **
assistant: KEYWORDS: country, Debel, located
**************************************************


Extracted keywords: ['Debel', 'country', 'located']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
['Lebanon (2009 film)', 'director', 'Samuel Maoz']
['Lebanon (2009 film)', 'country', 'Israel']
['Lebanon (2009 film)', 'international title', "Lebanon: The Soldier's Journey in the UK"]
['Lebanon (2009 film)', 'nationality'

2024-02-17 21:23:57,700 - httpx - INFO - HTTP Request: POST http://localhost:8111/chat/completions "HTTP/1.1 200 OK"
2024-02-17 21:23:57,715 - httpx - INFO - HTTP Request: POST http://localhost:8111/chat/completions "HTTP/1.1 200 OK"
2024-02-17 21:23:57,756 - bellek.llama_index.indices.knowledge_graph.retrievers - WARNING - Index was not constructed with embeddings, skipping embedding usage...


** Messages: **
system: You are an expert Q&A system that is trusted around the world.
Always answer the query using the provided context information, and not prior knowledge.
Some rules to follow:
1. Never directly reference the given context in your answer.
2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines.
user: Context information is below.
---------------------
kg_schema: {'schema': None}

The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
['Lebanon (2009 film)', 'director', 'Samuel Maoz']
['Lebanon (2009 film)', 'country', 'Israel']
['Lebanon (2009 film)', 'international title', "Lebanon: The Soldier's Journey in the UK"]
['Lebanon (2009 film)', 'nationality', 'Israeli']
['Lebanon (2009 film)', 'nominated for', 'Israeli Film Academy Award for Best Picture']
['Lebanon (2009 film)', 'status', 'Controver

2024-02-17 21:23:58,162 - httpx - INFO - HTTP Request: POST http://localhost:8111/chat/completions "HTTP/1.1 200 OK"


** Messages: **
system: You are an expert Q&A system that is trusted around the world.
Always answer the query using the provided context information, and not prior knowledge.
Some rules to follow:
1. Never directly reference the given context in your answer.
2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines.
user: Context information is below.
---------------------
kg_schema: {'schema': None}

The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
['Debel, Lebanon', 'country', 'Lebanon']
['Debel, Lebanon', 'is part of', 'Bint Jbeil']
['Debel, Lebanon', 'is part of', 'Nabatiye Governorate']
['Debel, Lebanon', 'is part of', 'Nabatiye, Lebanon']
['Debel, Lebanon', 'alternative name', 'Dibil']
['Debel, Lebanon', 'language', 'Arabic']
['66th Venice International Film Festival', 'award', "Leone d'Oro"]
['Lebanon (20

In [12]:

index.storage_context.index_store.index_structs?

Signature: index.storage_context.index_store.index_structs() -> List[llama_index.data_structs.data_structs.IndexStruct]
Docstring:
Get all index structs.

Returns:
    List[IndexStruct]: index structs
File:      ~/dev/repos/bellek/.venv/lib/python3.10/site-packages/llama_index/storage/index_store/keyval_index_store.py
Type:      method

In [13]:
index.storage_context.index_store.index_structs()[0]

KG(index_id='dc5463d2-b161-4270-be58-3458231e8866', summary=None, table={'14th Annual Satyajit Ray Award': {'4c90ff18-35c2-4265-bf5a-5b62e00782e6'}, 'Lebanon (2009 film)': {'4c90ff18-35c2-4265-bf5a-5b62e00782e6'}, '66th Venice International Film Festival': {'4c90ff18-35c2-4265-bf5a-5b62e00782e6'}, "Leone d'Oro": {'4c90ff18-35c2-4265-bf5a-5b62e00782e6'}, 'Samuel Maoz': {'4c90ff18-35c2-4265-bf5a-5b62e00782e6'}, 'Israel': {'4c90ff18-35c2-4265-bf5a-5b62e00782e6'}, "Lebanon: The Soldier's Journey in the UK": {'4c90ff18-35c2-4265-bf5a-5b62e00782e6'}, 'Israeli': {'4c90ff18-35c2-4265-bf5a-5b62e00782e6'}, 'Israeli Film Academy Award for Best Picture': {'4c90ff18-35c2-4265-bf5a-5b62e00782e6'}, 'Controversial': {'4c90ff18-35c2-4265-bf5a-5b62e00782e6'}, 'Debel, Lebanon': {'2e989546-f800-454f-8553-bbd7db068b69'}, 'Lebanon': {'2e989546-f800-454f-8553-bbd7db068b69'}, 'Bint Jbeil': {'2e989546-f800-454f-8553-bbd7db068b69'}, 'Nabatiye Governorate': {'2e989546-f800-454f-8553-bbd7db068b69'}, 'Nabatiye, Le

In [14]:
index._index_struct.__class__

bellek.llama_index.data_structs.data_structs.KG

In [15]:
query_engine.retriever._index_struct.__class__

bellek.llama_index.data_structs.data_structs.KG

In [16]:
storage_context = load_storage_context(knowledge_graph_directory / example_id)
storage_context

StorageContext(docstore=<llama_index.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x150a748e0>, index_store=<llama_index.storage.index_store.simple_index_store.SimpleIndexStore object at 0x150a74f10>, vector_stores={'image': <llama_index.vector_stores.simple.SimpleVectorStore object at 0x1530af460>, 'default': <llama_index.vector_stores.simple.SimpleVectorStore object at 0x150a47af0>}, graph_store=<bellek.llama_index.graph_stores.kuzu.KuzuGraphStore object at 0x150a74d30>)

In [17]:
storage_context.graph_store.get_rel_map()

{'Debel, Lebanon': [['Debel, Lebanon', 'country', 'Lebanon'],
  ['Debel, Lebanon', 'is part of', 'Bint Jbeil'],
  ['Debel, Lebanon', 'is part of', 'Nabatiye Governorate'],
  ['Debel, Lebanon', 'is part of', 'Nabatiye, Lebanon'],
  ['Debel, Lebanon', 'alternative name', 'Dibil'],
  ['Debel, Lebanon', 'language', 'Arabic']],
 'Lebanon (2009 film)': [['Lebanon (2009 film)', 'director', 'Samuel Maoz'],
  ['Lebanon (2009 film)', 'country', 'Israel'],
  ['Lebanon (2009 film)',
   'international title',
   "Lebanon: The Soldier's Journey in the UK"],
  ['Lebanon (2009 film)', 'nationality', 'Israeli'],
  ['Lebanon (2009 film)',
   'nominated for',
   'Israeli Film Academy Award for Best Picture'],
  ['Lebanon (2009 film)', 'status', 'Controversial']],
 '66th Venice International Film Festival': [['66th Venice International Film Festival',
   'award',
   "Leone d'Oro"]],
 '14th Annual Satyajit Ray Award': [['14th Annual Satyajit Ray Award',
   'winner',
   'Lebanon (2009 film)'],
  ['14th Annu

In [18]:
query_engine = index.as_query_engine(
    include_text=False,
    embedding_mode="hybrid",
    response_mode="simple_summarize",
    verbose=True,
)

In [19]:
sub_questions = [item["question"] for item in example["question_decomposition"]]
sub_questions

['In which country is Debel located?',
 'Who is the screenwriter of the film named after #1?']

In [20]:
example_answered = answer_questions(query_engine, example)
example_answered

2024-02-17 21:23:58,892 - httpx - INFO - HTTP Request: POST http://localhost:8111/chat/completions "HTTP/1.1 200 OK"
2024-02-17 21:23:58,945 - bellek.llama_index.indices.knowledge_graph.retrievers - WARNING - Index was not constructed with embeddings, skipping embedding usage...


** Messages: **
user: A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.
---------------------
In which country is Debel located?
---------------------
Provide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'

**************************************************
** Response: **
assistant: KEYWORDS: country, Debel, located
**************************************************


Extracted keywords: ['Debel', 'country', 'located']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
['Lebanon (2009 film)', 'director', 'Samuel Maoz']
['Lebanon (2009 film)', 'country', 'Israel']
['Lebanon (2009 film)', 'international title', "Lebanon: The Soldier's Journey in the UK"]
['Lebanon (2009 film)', 'nationality'

2024-02-17 21:23:59,499 - httpx - INFO - HTTP Request: POST http://localhost:8111/chat/completions "HTTP/1.1 200 OK"


** Messages: **
system: You are an expert Q&A system that is trusted around the world.
Always answer the query using the provided context information, and not prior knowledge.
Some rules to follow:
1. Never directly reference the given context in your answer.
2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines.
user: Context information is below.
---------------------
kg_schema: {'schema': None}

The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
['Lebanon (2009 film)', 'director', 'Samuel Maoz']
['Lebanon (2009 film)', 'country', 'Israel']
['Lebanon (2009 film)', 'international title', "Lebanon: The Soldier's Journey in the UK"]
['Lebanon (2009 film)', 'nationality', 'Israeli']
['Lebanon (2009 film)', 'nominated for', 'Israeli Film Academy Award for Best Picture']
['Lebanon (2009 film)', 'status', 'Controver

2024-02-17 21:24:00,079 - httpx - INFO - HTTP Request: POST http://localhost:8111/chat/completions "HTTP/1.1 200 OK"
2024-02-17 21:24:00,134 - bellek.llama_index.indices.knowledge_graph.retrievers - WARNING - Index was not constructed with embeddings, skipping embedding usage...


** Messages: **
user: A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.
---------------------
Who is the screenwriter of the film named after Debel is located in Lebanon.?
---------------------
Provide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'

**************************************************
** Response: **
assistant: KEYWORDS: screenwriter, film, Debel, Lebanon
**************************************************


Extracted keywords: ['film', 'Debel', 'Lebanon', 'screenwriter']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
['14th Annual Satyajit Ray Award', 'winner', 'Lebanon (2009 film)']
['14th Annual Satyajit Ray Award', 'winner', 'Lebanon (2009 film)', 'director', 'Samuel M

2024-02-17 21:24:00,835 - httpx - INFO - HTTP Request: POST http://localhost:8111/chat/completions "HTTP/1.1 200 OK"


** Messages: **
system: You are an expert Q&A system that is trusted around the world.
Always answer the query using the provided context information, and not prior knowledge.
Some rules to follow:
1. Never directly reference the given context in your answer.
2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines.
user: Context information is below.
---------------------
kg_schema: {'schema': None}

The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
['14th Annual Satyajit Ray Award', 'winner', 'Lebanon (2009 film)']
['14th Annual Satyajit Ray Award', 'winner', 'Lebanon (2009 film)', 'director', 'Samuel Maoz']
['14th Annual Satyajit Ray Award', 'winner', 'Lebanon (2009 film)', 'country', 'Israel']
['14th Annual Satyajit Ray Award', 'winner', 'Lebanon (2009 film)', 'international title', "Lebanon: The Soldier's Jo

{'id': '2hop__354073_511430',
 'question': 'Who is the screenwriter of the film named after the country Debel is located?',
 'question_decomposition': [{'question': 'In which country is Debel located?',
   'answer': 'Debel is located in Lebanon.'},
  {'question': 'Who is the screenwriter of the film named after #1?',
   'answer': 'The screenwriter of the film named after Debel, which is located in Lebanon, is not provided in the given context information.'}]}

In [21]:
query_engine = index.as_query_engine(
    include_text=False, 
    response_mode="tree_summarize",
    verbose=True,
)
response = query_engine.query(sub_questions[0])
response

2024-02-17 21:24:00,872 - httpx - INFO - HTTP Request: POST http://localhost:8111/chat/completions "HTTP/1.1 200 OK"
2024-02-17 21:24:01,012 - bellek.llama_index.indices.knowledge_graph.retrievers - WARNING - Index was not constructed with embeddings, skipping embedding usage...


** Messages: **
user: A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.
---------------------
In which country is Debel located?
---------------------
Provide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'

**************************************************
** Response: **
assistant: KEYWORDS: country, Debel, located
**************************************************


Extracted keywords: ['Debel', 'country', 'located']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
['Lebanon (2009 film)', 'director', 'Samuel Maoz']
['Lebanon (2009 film)', 'country', 'Israel']
['Lebanon (2009 film)', 'international title', "Lebanon: The Soldier's Journey in the UK"]
['Lebanon (2009 film)', 'nationality'

2024-02-17 21:24:01,487 - httpx - INFO - HTTP Request: POST http://localhost:8111/chat/completions "HTTP/1.1 200 OK"


** Messages: **
system: You are an expert Q&A system that is trusted around the world.
Always answer the query using the provided context information, and not prior knowledge.
Some rules to follow:
1. Never directly reference the given context in your answer.
2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines.
user: Context information from multiple sources is below.
---------------------
kg_schema: {'schema': None}

The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
['Lebanon (2009 film)', 'director', 'Samuel Maoz']
['Lebanon (2009 film)', 'country', 'Israel']
['Lebanon (2009 film)', 'international title', "Lebanon: The Soldier's Journey in the UK"]
['Lebanon (2009 film)', 'nationality', 'Israeli']
['Lebanon (2009 film)', 'nominated for', 'Israeli Film Academy Award for Best Picture']
['Lebanon (2009 film)'

Response(response='Debel is located in Lebanon.', source_nodes=[NodeWithScore(node=TextNode(id_='eca4a98f-783f-4da3-a9bc-f0dcb1a4328a', embedding=None, metadata={'kg_rel_texts': ["['Lebanon (2009 film)', 'director', 'Samuel Maoz']", "['Lebanon (2009 film)', 'country', 'Israel']", '[\'Lebanon (2009 film)\', \'international title\', "Lebanon: The Soldier\'s Journey in the UK"]', "['Lebanon (2009 film)', 'nationality', 'Israeli']", "['Lebanon (2009 film)', 'nominated for', 'Israeli Film Academy Award for Best Picture']", "['Lebanon (2009 film)', 'status', 'Controversial']", "['14th Annual Satyajit Ray Award', 'winner', 'Lebanon (2009 film)']", "['14th Annual Satyajit Ray Award', 'winner', 'Lebanon (2009 film)', 'director', 'Samuel Maoz']", "['14th Annual Satyajit Ray Award', 'winner', 'Lebanon (2009 film)', 'country', 'Israel']", '[\'14th Annual Satyajit Ray Award\', \'winner\', \'Lebanon (2009 film)\', \'international title\', "Lebanon: The Soldier\'s Journey in the UK"]', "['14th Annual

In [22]:
from llama_index.indices.knowledge_graph.retrievers import KGRetrieverMode

retriever = index.as_retriever(
    retriever_mode=KGRetrieverMode.HYBRID,
    include_text=False,
    verbose=True,
)

In [23]:
from llama_index.schema import QueryBundle

retriever.retrieve(QueryBundle(query_str=sub_questions[0]))

2024-02-17 21:24:01,639 - httpx - INFO - HTTP Request: POST http://localhost:8111/chat/completions "HTTP/1.1 200 OK"
2024-02-17 21:24:01,671 - bellek.llama_index.indices.knowledge_graph.retrievers - WARNING - Index was not constructed with embeddings, skipping embedding usage...


** Messages: **
user: A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.
---------------------
In which country is Debel located?
---------------------
Provide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'

**************************************************
** Response: **
assistant: KEYWORDS: country, Debel, located
**************************************************


Extracted keywords: ['Debel', 'country', 'located']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
['14th Annual Satyajit Ray Award', 'winner', 'Lebanon (2009 film)', 'international title', "Lebanon: The Soldier's Journey in the UK"]
['14th Annual Satyajit Ray Award', 'winner', 'Lebanon (2009 film)', 'nominated for', 'Isr

[NodeWithScore(node=TextNode(id_='9f028baa-f531-4598-a8c5-001d87ef0cbf', embedding=None, metadata={'kg_rel_texts': ['[\'14th Annual Satyajit Ray Award\', \'winner\', \'Lebanon (2009 film)\', \'international title\', "Lebanon: The Soldier\'s Journey in the UK"]', "['14th Annual Satyajit Ray Award', 'winner', 'Lebanon (2009 film)', 'nominated for', 'Israeli Film Academy Award for Best Picture']", "['14th Annual Satyajit Ray Award', 'winner', 'Lebanon (2009 film)', 'status', 'Controversial']", "['14th Annual Satyajit Ray Award', 'winner', 'Lebanon (2009 film)', 'director', 'Samuel Maoz']", "['14th Annual Satyajit Ray Award', 'winner', 'Lebanon (2009 film)', 'nationality', 'Israeli']", '[\'Lebanon (2009 film)\', \'international title\', "Lebanon: The Soldier\'s Journey in the UK"]', "['14th Annual Satyajit Ray Award', 'winner', 'Lebanon (2009 film)', 'country', 'Israel']", "['Lebanon (2009 film)', 'nominated for', 'Israeli Film Academy Award for Best Picture']", '[\'66th Venice Internation

In [24]:
retriever._index_struct.search_node_by_keyword("Midland, Texas")

{}

In [25]:
index.storage_context.graph_store.get_rel_map()

{'Debel, Lebanon': [['Debel, Lebanon', 'country', 'Lebanon'],
  ['Debel, Lebanon', 'is part of', 'Bint Jbeil'],
  ['Debel, Lebanon', 'is part of', 'Nabatiye Governorate'],
  ['Debel, Lebanon', 'is part of', 'Nabatiye, Lebanon'],
  ['Debel, Lebanon', 'alternative name', 'Dibil'],
  ['Debel, Lebanon', 'language', 'Arabic']],
 '14th Annual Satyajit Ray Award': [['14th Annual Satyajit Ray Award',
   'winner',
   'Lebanon (2009 film)'],
  ['14th Annual Satyajit Ray Award',
   'winner',
   'Lebanon (2009 film)',
   'director',
   'Samuel Maoz'],
  ['14th Annual Satyajit Ray Award',
   'winner',
   'Lebanon (2009 film)',
   'country',
   'Israel'],
  ['14th Annual Satyajit Ray Award',
   'winner',
   'Lebanon (2009 film)',
   'international title',
   "Lebanon: The Soldier's Journey in the UK"],
  ['14th Annual Satyajit Ray Award',
   'winner',
   'Lebanon (2009 film)',
   'nationality',
   'Israeli'],
  ['14th Annual Satyajit Ray Award',
   'winner',
   'Lebanon (2009 film)',
   'nominated f

In [26]:
index.storage_context.graph_store.get("Midland, Texas")

[]